<a href="https://colab.research.google.com/github/GolfSuradej/IrisPAD/blob/main/Iris_2D3D_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import os
import numpy as np
from google.colab import drive
import h5py
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

Import Data

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
cd /content/drive/MyDrive/Seniorsource

/content/drive/MyDrive/Seniorsource


In [4]:
ls

fakemyBISF.npy  LivDet2013/  realmyBISF.npy  Test_bona.npy  Test_Fake.npy


BSIF Feature Extration

In [5]:
trainFeature_bona = np.load("realmyBISF.npy")
trainFeature_fake = np.load("fakemyBISF.npy")
trainLabel_bona = np.zeros([len(trainFeature_bona),1])
trainLabel_fake = np.ones([len(trainFeature_fake),1])

In [6]:
testFeature_bona = np.load("Test_bona.npy")
testFeature_fake = np.load("Test_Fake.npy")
testLabel_bona = np.zeros([len(testFeature_bona),1])
testLabel_fake = np.ones([len(testFeature_fake),1])

In [7]:
print("No. of the training samples (Bona) : " ,len(trainFeature_bona))
print("No. of the training samples (Fake) : " ,len(trainFeature_fake))
print("No. of the test samples (Bona) : " ,len(testFeature_bona))
print("No. of the test samples (Fake) : " ,len(testFeature_fake))

No. of the training samples (Bona) :  270
No. of the training samples (Fake) :  400
No. of the test samples (Bona) :  246
No. of the test samples (Fake) :  440


In [9]:
print("BISF feature: " ,np.transpose(trainFeature_bona[0]))

BISF feature:  [[110112.  12212.  18191.   9758.   8275.  14248.  10492.  11970.   9803.
   20004.  11646.  14103.   9246.  14224.   9791.  11233.   7907.  10107.
   10663.   9416.   9293.   9761.  13880.   7589.  12222.  13856.  13782.
   11754.   7896.  14601.   9458.  10507.]]


Prepare training data

In [10]:
X = np.vstack((trainFeature_bona,trainFeature_fake))
Y = np.vstack((trainLabel_bona,trainLabel_fake))
print(X.shape)
print(Y.shape)

(670, 32, 1)
(670, 1)


In [11]:
X_train, Y_train = shuffle(X,Y)
np.save('X_train',X_train)
np.save('Y_train',Y_train)

In [16]:
x = X_train.reshape(len(X),32)
y = Y_train.reshape(len(Y),1)
print(x.shape)
print(y.shape)

(670, 32)
(670, 1)


SVM

In [12]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score

In [13]:
clf = svm.SVC(kernel='linear') # instantiate classifier with default hyperparameters

Train or fitting the model

In [17]:
clf.fit(x, y)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='linear')

In [18]:
import joblib
joblib.dump(clf,"SVM_linear.pkl")  #save model

['SVM_linear.pkl']

In [20]:
clf.predict(testFeature_bona[0].reshape(1,32))

array([0.])

In [21]:
clf.predict(testFeature_fake[1].reshape(1,32))

array([1.])

In [22]:
testBISF = np.vstack((testFeature_bona,testFeature_fake))
testLabel = np.vstack((testLabel_bona,testLabel_fake))

In [24]:
predicted = np.zeros([len(testBISF),1])
for i in range(len(testBISF)):
  predicted[i] = clf.predict(testBISF[i].reshape(1,32))

In [25]:
accuracy_score(testLabel, predicted)

0.7536443148688047

prob

In [39]:
SVMprob =  svm.SVC(kernel='linear',probability=True)
SVMprob.fit(x, y)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='linear', probability=True)

In [40]:
joblib.dump(SVMprob,"SVM_linearProb2.pkl")  #save model

['SVM_linearProb2.pkl']

In [42]:
print(SVMprob)

SVC(kernel='linear', probability=True)


In [43]:
SVMprob.predict_proba(testBISF[0].reshape(1,32))

array([[0.92601619, 0.07398381]])

In [32]:
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_score

In [33]:
fpr, tpr, thresholds = roc_curve(predicted, testLabel, pos_label=1)

def compute_eer(fpr,tpr,thresholds):
    """ Returns equal error rate (EER) and the corresponding threshold. """
    fnr = 1-tpr
    print(fnr)
    abs_diffs = np.abs(fpr - fnr)
    min_index = np.argmin(abs_diffs)
    eer = np.mean((fpr[min_index], fnr[min_index]))
    return eer, thresholds[min_index]
    
err,thr = compute_eer(fpr, tpr, thresholds)
print(err)
print(thr)

[1.         0.19686801 0.        ]
0.26789007141987964
1.0


In [52]:
predict_class = []
for k in predictedProb:
  predict_class.append(k[1] - k[0])

In [44]:
predictedProb = np.zeros([len(testBISF),2])
for i in range(len(testBISF)):
  predictedProb[i] = SVMprob.predict_proba(testBISF[i].reshape(1,32))

In [48]:
fpr, tpr, thresholds = roc_curve(testLabel, predict_class, pos_label=1)

def compute_eer(fpr,tpr,thresholds):
    """ Returns equal error rate (EER) and the corresponding threshold. """
    fnr = 1-tpr
    abs_diffs = np.abs(fpr - fnr)
    min_index = np.argmin(abs_diffs)
    eer = np.mean((fpr[min_index], fnr[min_index]))
    return eer, thresholds[min_index]
    
err,thr = compute_eer(fpr, tpr, thresholds)


0.28432187730968217
0.4745805282295053


In [49]:
print("EER = ", err)
print("Threshold @ ",thr)

EER =  0.28432187730968217
Threshold@  0.4745805282295053
